In [1]:
import pandas as pd
import importlib

In [2]:
dataDir = '/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/2023_03_01_v0_dataGathering/PoreC_Fibroblasts/'
chromSizes = pd.read_csv(f'{dataDir}hg38.chromSizes',sep="\t", names = ['chr','size']).set_index('chr')['size'].to_dict()
readConcatemersWClosestGene = f'{dataDir}neonatal_fragsOutput_byChr/neonatal_fibroblasts_chr2.gz'
colnames = ["chr","start","end","readID","readLen","readQual",
"geneChr","geneStart","geneEnd","strand","geneID","bioType","geneName","dist","ID"]

fullBed = pd.read_csv(readConcatemersWClosestGene,sep = "\t",names = colnames)

In [3]:
chr19 = fullBed[fullBed['chr']=="chr2"]
binSize = 1*10**6 #5*10**5
chrBins = [x for x in range(0,chromSizes['chr2']+binSize,binSize)]
chr19_binned = pd.cut(chr19['start'],bins = chrBins, labels = ["Bin"+str(i+1) for i in range(len(chrBins)-1)]).rename("binID")
chr19_wBinID = chr19.merge(chr19_binned,left_index=True,right_index=True)

In [4]:
chr19_wBinID[['ID','binID']].tail()

,ID,binID
612155,382207,Bin242
612156,382208,Bin242
612157,382045,Bin243
612158,382209,Bin243
612159,382210,Bin243


In [58]:
groupedBins = chr19_wBinID.groupby('ID')['binID'].apply(list).reset_index(name='Bins')

In [59]:
def sort_key(item):
    return int(item.split('Bin')[1])

edges = ["_".join(sorted(list(set(a)), key=sort_key)) for a in groupedBins['Bins'] if len(list(set(a))) > 1]
readIDs = [groupedBins.iloc[ix][0] for ix in range(len(groupedBins)) if len(list(set(groupedBins.iloc[ix][1]))) > 1]
print(len(edges))
print(len(readIDs))

105355
105355


In [60]:
from collections import Counter
edgeDict = dict(Counter(edges))


In [73]:
len(edgeDict.keys())

57618

In [62]:
import pickle

with open(f'{dataDir}adult_chr2_dict.pkl','wb') as f:
    pickle.dump(edgeDict,f)

In [6]:
import sys
sys.path.append('/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/v0.analysis/scripts/pythonScripts/functions/')
from promethData_multiwayExpectedProbs import multiwayEval_realData


In [50]:
dataDir = '/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/2023_03_01_v0_dataGathering/PoreC_Fibroblasts/'
runDir = 'v1.evaluateExpectedVersusInteresting_adult/'
plotDir = f'{dataDir}{runDir}Plots_adult_chr2/'
outDir = f'{dataDir}{runDir}dfs_adult_chr2/'
inputPkl = 'adult_chr2_dict.pkl'
probHashOutName = f'{outDir}probHash_chr2.json'

seed = 1
quartile = 25
toChoose = 400
toPlotRef = False
toPlotInd = False
toPlotScatter = False

In [51]:
import os
import json
import warnings
import pickle

print("Loading in hyperedge pickle file")
with open(f'{dataDir}/{inputPkl}','rb') as f:
    hpEdges = pickle.load(f)

print("Processing all the hyperedges from pickle file")
hpKeys = [k for k in hpEdges.keys()]
print("A total of",len(hpKeys),"initial interactions")

readSupport = [v for v in hpEdges.values()]
atLeastTwoChains = [i for i,x in enumerate(readSupport) if x >=2]
updatedDict = {hpKeys[i]:readSupport[i] for i in atLeastTwoChains}

hpKeys = [k for k in updatedDict.keys()]
hpKeys_split = [k.split("_") for k in updatedDict.keys()]
keyCard = [len(item) for item in hpKeys_split]

print("Updated the input: retaining",len(hpKeys),
        "interactions with chain support of at least 2")

evalInstance = multiwayEval_realData(keyCard, updatedDict, hpKeys, hpKeys_split, seed,
                            toChoose, toPlotRef, toPlotInd, toPlotScatter,
                            quartile, plotDir,outDir)

if os.path.exists(probHashOutName):
    print("Expected probabilities file already exists...moving on")
    with open(probHashOutName,'r') as file:
        tmpHash = json.load(file)
        # probHash = {key: {int(k): v for k, v in value.items() if value is not None} 
        #             if value is not None else None for key, value in tmpHash.items()}
        probHash = {key: {int(k): v for k, v in value.items()} for key, value in tmpHash.items() if value is not None}

else:
    print("Expected probabilities file does not exist...creating now")
    probHash = evalInstance.makeAllReferenceHashDicts()
    with open(probHashOutName,'w') as file:
        json.dump(probHash,file)
    print("File created...moving on")

warnings.filterwarnings('ignore')


Loading in hyperedge pickle file
Processing all the hyperedges from pickle file
A total of 57618 initial interactions
Updated the input: retaining 16581 interactions with chain support of at least 2
Expected probabilities file does not exist...creating now
Calculating for card= 8
There are  2 reads
Creating a probability hash for 2-way subsets
Creating a probability hash for 3-way subsets
Creating a probability hash for 4-way subsets
Creating a probability hash for 5-way subsets
Creating a probability hash for 6-way subsets
Creating a probability hash for 7-way subsets
Calculating for card= 7
There are  1 reads
Creating a probability hash for 2-way subsets
Creating a probability hash for 3-way subsets
Creating a probability hash for 4-way subsets
Creating a probability hash for 5-way subsets
Creating a probability hash for 6-way subsets
Calculating for card= 6
There are  4 reads
Creating a probability hash for 2-way subsets
Creating a probability hash for 3-way subsets
Creating a proba

In [28]:
evalInstance = multiwayEval_realData(keyCard, updatedDict, hpKeys, hpKeys_split, seed,
                            toChoose, toPlotRef, toPlotInd, toPlotScatter,
                            quartile, plotDir,outDir)

In [30]:
from collections import Counter
# dict(Counter(keyCard))

In [52]:
import importlib
import promethData_multiwayExpectedProbs
importlib.reload(promethData_multiwayExpectedProbs)

from promethData_multiwayExpectedProbs import multiwayEval_realData

In [53]:
toChoose = 1000

evalInstance = multiwayEval_realData(keyCard, updatedDict, hpKeys, hpKeys_split, seed,
                            toChoose, toPlotRef, toPlotInd, toPlotScatter,
                            quartile, plotDir,outDir)

A = evalInstance.statsForAllReads(probHash)

# card = 6
# ixList = [index for index,element in enumerate(keyCard) if element == card]
# print(len(ixList))

# evalInstance.getReadExpectednessStats(6,ixList[2],3,probHash['6sub3'])
# evalInstance.getStatsPerCard(6,3,probHash,ixList[0:10])
# for n in range(2,card):
#     stats = evalInstance.getStatsPerCard(card,n,probHash,ixList[0:10])
#     print(stats)
# evalInstance.statsForAllCardSubsets(10,probHash)

Calculating for card= 8
There are  2 reads
Calculating stats for 2 reads
Writing output for wDist
Writing output for cosineSim
Writing output for eDist
Writing output for empDist
Calculating for card= 7
There are  1 reads
Calculating for card= 6
There are  4 reads
Calculating stats for 4 reads
Writing output for wDist
Writing output for cosineSim
Writing output for eDist
Writing output for empDist
Calculating for card= 5
There are  12 reads
Calculating stats for 12 reads
Writing output for wDist
Writing output for cosineSim
Writing output for eDist
Writing output for empDist
Calculating for card= 4
There are  45 reads
Calculating stats for 45 reads
Writing output for wDist
Writing output for cosineSim
Writing output for eDist
Writing output for empDist
Calculating for card= 3
There are  938 reads
Calculating stats for 938 reads
Writing output for wDist
Writing output for cosineSim
Writing output for eDist
Writing output for empDist


## RNA-Seq data 

In [14]:
filepath = '/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/2023_03_01_v0_dataGathering/PoreC_Fibroblasts/adult_All_ONT_RNA_fromGEO.mat'
import numpy as np
import scipy.io
import pandas as pd

arrays = {}
f = scipy.io.loadmat(filepath)
for k, v in f.items():
    arrays[k] = np.array(v)

In [6]:
arrays

{'__header__': array(b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Nov 24 09:59:45 2021',
       dtype='|S76'),
 '__version__': array('1.0', dtype='<U3'),
 '__globals__': array([], dtype=float64),
 'None': array([(b'All_ONT_RNA_20210719', b'MCOS', b'table', array([[3707764736],
               [         2],
               [         1],
               [         1],
               [         1],
               [         1]], dtype=uint32))                             ],
       dtype=[('s0', 'O'), ('s1', 'O'), ('s2', 'O'), ('arr', 'O')]),
 '__function_workspace__': array([[ 0,  1, 73, ...,  0,  0,  0]], dtype=uint8)}